<a href="https://colab.research.google.com/github/godinezseba/Tarea-2-RedesNeuronales/blob/master/notebooks/Parte%202/pruebas/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/drive/1ixOZTKLz4aAa-MtC6dy_sAvc9HujQmHN#scrollTo=szoSlW1HGKWI

In [ ]:
!git clone https://github.com/godinezseba/Tarea-2-RedesNeuronales

fatal: destination path 'Tarea-2-RedesNeuronales' already exists and is not an empty directory.


In [ ]:
!pip3 install ktrain

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Conv1D, Flatten, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from keras.initializers import Constant
from sklearn.metrics import  classification_report
import re

Using TensorFlow backend.


In [ ]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

In [ ]:
!pip install tokenizers

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
import os
import re

In [ ]:
max_len = 384
configuration = BertConfig()

#Cargar datos

In [ ]:
train = pd.read_csv('./Tarea-2-RedesNeuronales/datasets/Parte 2/train_source_tweets.txt', sep='\t', names = ['ID','Tweet'] )

In [ ]:
labels = pd.read_csv('./Tarea-2-RedesNeuronales/datasets/Parte 2/train_labels.csv' )

In [ ]:
test = pd.read_csv('./Tarea-2-RedesNeuronales/datasets/Parte 2/test_source_tweets.txt', sep='\t', names = ['ID','Tweet'] )

In [ ]:
print(labels['label'].value_counts())

non-rumor     524
false         520
true          514
unverified    368
Name: label, dtype: int64


In [ ]:
x_train = train['Tweet'].values
y_train = labels['label'].values
x_test = test['Tweet'].values



# Pre procesar

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train, x_test=x_test,
                                                                       class_names=['non-rumor','false','true','unverified'],
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

task: text classification
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:521: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


#BERT

In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Train on 1733 samples
Epoch 1/4
1733/1733 [==============================] - 164s 95ms/sample - loss: 1.3417 - accuracy: 0.3526
Epoch 2/4
1733/1733 [==============================] - 149s 86ms/sample - loss: 0.8906 - accuracy: 0.6469
Epoch 3/4
1733/1733 [==============================] - 149s 86ms/sample - loss: 0.3552 - accuracy: 0.8892
Epoch 4/4
1733/1733 [==============================] - 149s 86ms/sample - loss: 0.0956 - accuracy: 0.9769


#Kaggle

In [ ]:
le = LabelEncoder()

qwer = le.fit_transform(labels['label'])

In [ ]:
x_test2 = test['Tweet'].values

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()

['false', 'non-rumor', 'true', 'unverified']

In [ ]:
pred = predictor.predict(x_test2)

In [ ]:
print(pred)

['unverified', 'unverified', 'false', 'true', 'false', 'false', 'false', 'false', 'true', 'unverified', 'unverified', 'non-rumor', 'false', 'non-rumor', 'true', 'false', 'non-rumor', 'true', 'true', 'false', 'false', 'unverified', 'true', 'false', 'non-rumor', 'true', 'false', 'false', 'true', 'non-rumor', 'non-rumor', 'non-rumor', 'false', 'unverified', 'false', 'non-rumor', 'true', 'true', 'true', 'true', 'non-rumor', 'unverified', 'true', 'non-rumor', 'true', 'true', 'true', 'false', 'true', 'true', 'false', 'false', 'non-rumor', 'true', 'false', 'true', 'true', 'false', 'non-rumor', 'true', 'false', 'false', 'false', 'non-rumor', 'true', 'true', 'unverified', 'true', 'non-rumor', 'true', 'false', 'non-rumor', 'true', 'true', 'unverified', 'non-rumor', 'false', 'true', 'true', 'true', 'unverified', 'true', 'true', 'false', 'true', 'non-rumor', 'false', 'false', 'non-rumor', 'non-rumor', 'true', 'unverified', 'false', 'unverified', 'unverified', 'non-rumor', 'non-rumor', 'non-rumor',

In [ ]:
asd = model.predict_classes(X)

In [ ]:
pred = le.inverse_transform(asd)

In [ ]:
sample = pd.read_csv('./Tarea-2-RedesNeuronales/datasets/Parte 2/sample_submission.csv')

sample['label'] = pred
sample['id'] = test['ID']
sample.to_csv('sample_cnn.csv', header=True, index=False)